In [1]:
import numpy as np
import pandas as pd

def split(s):
    try:
        splits = s.split('_')
        if len(splits) == 2:
            return pd.Series({'Genus_A':splits[0], 'Genus_B':splits[1]})
        else:
            if "Peptostreptococcaceae_incertae_sedis" in s:
                if s.startswith("Peptostreptococcaceae_incertae_sedis"):
                    return pd.Series({'Genus_A':"Peptostreptococcaceae_incertae_sedis", 
                                      'Genus_B':s.replace("Peptostreptococcaceae_incertae_sedis","")[1:]})
                else:
                    return pd.Series({'Genus_B':"Peptostreptococcaceae_incertae_sedis", 
                                      'Genus_A':s.replace("Peptostreptococcaceae_incertae_sedis","")[:-1]})
            if "Lachnospiracea_incertae_sedis" in s:
                if s.startswith("Lachnospiracea_incertae_sedis"):
                    return pd.Series({'Genus_A':"Lachnospiracea_incertae_sedis", 
                                      'Genus_B':s.replace("Lachnospiracea_incertae_sedis","")[1:]})
                else:
                    return pd.Series({'Genus_B':"Lachnospiracea_incertae_sedis", 
                                      'Genus_A':s.replace("Lachnospiracea_incertae_sedis","")[:-1]})    
    except:
        print("here")
        return None
    
df_summary = pd.read_csv('/Users/harihara/Mount-2/projects/RDP_Outlier_Analysis/\
16S_SILVA_Experimentally_Validated_Strains_Full_DB/Label_Changes.SILVA_Short.Summary', sep = "\t")
seq_path = '/Users/harihara/Mount-2/projects/RDP_Outlier_Analysis/\
16S_SILVA_Experimentally_Validated_Strains_Full_DB/Labelled_16S_Sequences.fasta'
SILVA_taxa = {}
for l in open(seq_path).readlines():
    if l.startswith('>'):
        l = l.replace(">","").replace("\n","")
        seq_id = l.split(' ')[0].split(".")[0]
        try: taxa = l.split(' ')[1].split(";")[5]
        except IndexError: taxa = ""
        SILVA_taxa[seq_id] = taxa
df_SILVA_taxa = pd.DataFrame(data = {'Seq_ID':SILVA_taxa.keys(), 'Genus':SILVA_taxa.values()})


In [2]:
df_merged = df_summary.merge(df_SILVA_taxa, left_on = 'Accession', 
                             right_on = 'Seq_ID', how = 'left')\
                      .drop(columns = ['Seq_ID'])\
                      .rename(columns = {'Genus':'Genus_Query(SILVA)'})
df_merged = df_merged.merge(df_SILVA_taxa, left_on = 'GA', 
                            right_on = 'Seq_ID', how = 'left')\
                     .drop(columns = ['Seq_ID'])\
                     .rename(columns = {'Genus':'Genus_A(SILVA)'})
df_merged = df_merged.merge(df_SILVA_taxa, left_on = 'GB', 
                            right_on = 'Seq_ID', how = 'left')\
                     .drop(columns = ['Seq_ID'])\
                     .rename(columns = {'Genus':'Genus_B(SILVA)'})

df_merged[['Genus_A','Genus_B']] = df_merged['File'].apply(split)

In [3]:
df_filter = df_merged[(df_merged['Genus_Query(SILVA)'] == df_merged['G-WO_ADV'])&
                      (df_merged['Genus_A(SILVA)'] == df_merged['Genus_A'])&
                      (df_merged['Genus_B(SILVA)'] == df_merged['Genus_B'])]

In [4]:
sorted(list(set(list(zip(df_filter['G-WO_ADV'].tolist(), df_filter['G-ADV'].tolist())))))

[('Actinomadura', 'Actinoallomurus'),
 ('Afipia', 'Bradyrhizobium'),
 ('Afipia', 'Rhodopseudomonas'),
 ('Alkalispirillum', 'Alkalilimnicola'),
 ('Aquamicrobium', 'Pseudaminobacter'),
 ('Bradyrhizobium', 'Rhodopseudomonas'),
 ('Chryseoglobus', 'Microcella'),
 ('Citrobacter', 'Enterobacter'),
 ('Citrobacter', 'Klebsiella'),
 ('Citrobacter', 'Salmonella'),
 ('Colwellia', 'Thalassotalea'),
 ('Coprococcus', 'Mediterraneibacter'),
 ('Cronobacter', 'Buttiauxella'),
 ('Cronobacter', 'Klebsiella'),
 ('Cronobacter', 'Raoultella'),
 ('Cronobacter', 'Shimwellia'),
 ('Enterobacter', 'Leclercia'),
 ('Enterobacter', 'Raoultella'),
 ('Frigidibacter', 'Defluviimonas'),
 ('Halomonas', 'Chromohalobacter'),
 ('Halovarius', 'Natribaculum'),
 ('Humibacillus', 'Terrabacter'),
 ('Intestinibacter', 'Terrisporobacter'),
 ('Jannaschia', 'Leisingera'),
 ('Methyloterrigena', 'Paradevosia'),
 ('Microbacterium', 'Rathayibacter'),
 ('Nguyenibacter', 'Gluconacetobacter'),
 ('Paradevosia', 'Methyloterrigena'),
 ('Stark

In [71]:
len(df_filter)

1014

In [72]:
len(df_merged)

2599

In [6]:
len(df_filter['Accession'].unique())

53

In [9]:
len(set(df_filter['GA'].tolist() + df_filter['GB'].tolist()))

233

In [22]:
len(set(df_merged['GA'].tolist() + df_merged['GB'].tolist()))

564

In [13]:
len(df_merged['Accession'].unique())

107

In [18]:
df_merged['Y'] = df_merged['GA']+'_'+df_merged['GA']

In [21]:
len(df_merged['Y'].unique())

248

In [23]:
248*2

496

In [29]:
df_filter[df_filter['Genus_Query(SILVA)'] == 'Streptomyces']

,Accession,Seq-ID,G-ADV,G-WO_ADV,File,GA,GB,Genus_Query(SILVA),Genus_A(SILVA),Genus_B(SILVA),Genus_A,Genus_B
1770,HQ286045,HQ286045.1.1528,Prauserella,Streptomyces,Prauserella_Amycolatopsis,AF466190,AJ252832,Streptomyces,Prauserella,Amycolatopsis,Prauserella,Amycolatopsis
1771,HQ286045,HQ286045.1.1528,Prauserella,Streptomyces,Prauserella_Amycolatopsis,AF466190,CP002896,Streptomyces,Prauserella,Amycolatopsis,Prauserella,Amycolatopsis


In [26]:
df_filter['Genus_Query(SILVA)'].unique()

array(['Actinomadura', 'Afipia', 'Tardiphaga', 'Alkalispirillum',
       'Starkeya', 'Cronobacter', 'Halomonas', 'Chryseoglobus',
       'Citrobacter', 'Coprococcus', 'Frigidibacter', 'Enterobacter',
       'Nguyenibacter', 'Jannaschia', 'Paradevosia', 'Halovarius',
       'Methyloterrigena', 'Streptomyces', 'Aquamicrobium',
       'Microbacterium', 'Bradyrhizobium', 'Humibacillus',
       'Intestinibacter', 'Thalassobius', 'Colwellia'], dtype=object)